In [13]:
import pandas as pd
import datetime as dt

In [ ]:
import sys
from pathlib import Path

project_path = Path(r'D:\OneDrive - CELSIA S.A E.S.P') / 'Proyectos' / 'Eficiencia_Energetica' / 'Esenttia' / 'Reporte_2022_09_15'
sys.path.append(str(project_path))

In [14]:
dct_names = {
    'Planta1.13': 'Planta1.13',
    'Planta1.14': 'Planta1.14',
    'Planta1.15': 'Planta1.15',
    'Planta1.16': 'Planta1.16',
    'Planta1.17': 'Planta1.17',
    'Planta1.18': 'Planta1.18',
    'Planta1.19': 'Planta1.19',
    'Planta1.24': 'Planta1.24',
    'Planta1.29': 'Planta1.29',
    'Planta1.3': 'Planta1.3',
    'Planta1.30': 'Planta1.30',
    'Planta1.31': 'Planta1.31',
    'Planta1.32': 'Planta1.32',
    'Planta1.34': 'Planta1.34',
    'Planta1.35': 'Planta1.35',
    'Planta1.36': 'Planta1.36',
    'Planta1.37': 'Planta1.37',
    'Planta1.38': 'Planta1.38',
    'Planta1.39': 'Planta1.39',
    'Planta1.40': 'Planta1.40',
    'Planta1.5': 'Planta1.5',
    'Planta1.G0': 'Planta1.G0404',
    'Planta1.G2': 'Planta1.G2012',
    'Planta1.K0': 'Planta1.K0304',
    'Planta1.Y0': 'Planta1.Y0400F1M',
    'Planta2.1': 'Planta2.1',
    'Planta2.10': 'Planta2.10',
    'Planta2.11': 'Planta2.11',
    'Planta2.12': 'Planta2.12',
    'Planta2.2': 'Planta2.2',
    'Planta2.20': 'Planta2.20K10M01',
    'Planta2.21': 'Planta2.21',
    'Planta2.22': 'Planta2.22',
    'Planta2.26': 'Planta2.26',
    'Planta2.27': 'Planta2.27',
    'Planta2.33': 'Planta2.33',
    'Planta2.6': 'Planta2.6',
    'Planta2.8_': 'Planta2.8_01P010BM01',
    'Planta2.9': 'Planta2.9'
}

cols = [
    'name',
    'date',
    'time',
    'active_energy_delivered',
    'reactive_energy_delivered',
    'active_power',
    'apparent_power',
    'reactive_power',
    'power_factor_signed',
    'average_voltage_ltl',
    'average_voltage_ltn',
]

name_to_description = {
    '1':'Compresor de nitrógeno',
    '2':'Compresor gas transporte',
    '3':'Soplador aire de transporte',
    '5':'Bomba torre de enfriamiento PTA1',
    '6':'Bomba agua torres',
    '9':'Ventilador de descarga de aditivos sólidos',
    '10':'Equipo reemplazado',
    '11':'Ventilador de descarga de aditivos sólidos',
    '12':'Equipo mal rotulado',
    '13':'Soplador granel',
    '14':'Bomba calentador propileno - Compresor de nitrógeno',
    '15':'Bomba de propileno área 20',
    '16':'Bomba de propileno',
    '17':'Bomba calentador propileno',
    '18':'Bombas: Calentador propileno -  Calentador propileno',
    '19':'No registra',
    '21':'Bombas: Agua torres - Agua torres',
    '22':'Purga del agitador del reactor - mal rotulado',
    '24':'Motor de ventilador lado norte',
    '26':'Compresor de nitrógeno',
    '27':'Bomba de propileno',
    '29':'Ventilador',
    '30':'Bomba torre de enfriamiento PTA1',
    '31':'Bombas: Booster propileno - Intercambiador E-4002',
    '32':'Bombas: Agua tratada - Agua tratada',
    '33':'Bombas: Agua torres - Agua torres',
    '34':'No registra',
    '35':'Bomba booster de propileno',
    '36':'Fuera de servicio',
    '37':'Desconocido - Soplador para purgar silos',
    '38':'Sopladores: Pellets - Baggin/Granel - Baggin',
    '39':'Compresor de aire ZR250',
    '40':'Compresor de aire ZR275 FF servicos',
    '20K10M01':'Compresor de inertes',
    '8_01P010BM01':'Bomba de propileno',
    'G0404':'Bomba torre de enfriamiento PTA1',
    'G2012':'Bomba booster de propileno',
    'K0304':'Soplador de silos',
    'PM23':'Bomba torre de enfriamiento',
    'PM25':'Bomba torre de enfriamiento PTA1',
    'Y0400F1M':'Ventilador lado sur'
}

In [15]:
def prepare_data(df, dct_names, cols):
    # set local timezone DatetimeIndex
    df['datetime'] = pd.to_datetime(df['date'] + ' ' + df['time'])
    df.drop(columns=['date','time'], inplace=True)
    df.set_index('datetime', inplace=True)
    df = df.tz_localize('utc').tz_convert('America/Bogota')

    # melt to compress dataset and also make it easier to manipulate with our existing libraries
    df = df.reset_index().melt(id_vars=['name','datetime'], value_vars=cols[3:])

    # fix names
    df['name'] = df['name'].astype(str).map(dct_names)

    meters = pd.read_excel('Lista_medidores_esenttia.xlsx')
    meters['name'] = meters['group'] + '.' + meters['name'].astype(str)
    name_to_load = dict(zip(meters['name'], meters['load']))

    df['load'] = df['name'].map(name_to_load)
    df['description'] = df['name'].str.split('.', expand=True)[1].map(name_to_description)

    # df['group'] = df['name'].apply(lambda x: x.split('.')[0])
    # df['name'] = df['name'].apply(lambda x: x.split('.')[1])

    # configure dtypes
    df['name'] = df['name'].astype('category')
    df['description'] = df['description'].astype('category')
    df['variable'] = df['variable'].astype('category')
    df['load'] = df['load'].astype('category')
    df['value'] = df['value'].str.replace(',', '.').astype(float)

    # clean the data
    faulty_names = [
        '(2558588 r',
        'Completion',
        'Warning: N',
        'Planta1.34', # no associated load in the table provided by Diego
        'Planta1.37', # doesn't exist
        'Planta1.39', # failed
        'Planta1.40', # doesn't exist
        
    ]
    df = df[~df['name'].isin(faulty_names)].copy()
    df = df.dropna(how='any')
    df = df.drop_duplicates(subset=['datetime','name','variable'])

    df = df.set_index('datetime')
    df = df[df.index.year != 2018]

    return df

In [16]:
# df = pd.read_fwf('data_2022_09_13.txt', skiprows=2, header=None, names=cols)
# pd.to_pickle(df, 'raw_read_fwf.pkl')

df = pd.read_pickle('raw_read_fwf.pkl')

In [17]:
df = prepare_data(df, dct_names, cols)
df.dtypes

name           category
variable       category
value           float64
load           category
description    category
dtype: object

In [23]:
pd.to_pickle(df, project_path / 'data' / 'data_long.pkl')